In [ ]:
104

In [ ]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup

"""
設定查詢條件
"""
# 加入使用者資訊(如使用什麼瀏覽器、作業系統...等資訊)模擬真實瀏覽網頁的情況
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

# 查詢的關鍵字
my_params = {'ro':'1', # 限定全職的工作，如果不限定則輸入0
             'keyword':'數據分析', # 想要查詢的關鍵字
             'area':'6001001000%2C6001002003%2C6001002007%2C6001002011%2C6001002014%2C6001002015%2C6001002021%2C6001002020%2C6001002023%2C6001002025', # 限定在台北+新北(西部)的工作
             'isnew':'1', # 只要最近一個月有更新的過的職缺
             'isnew':'10', # 只要最近一個月有更新的過的職缺
             'mode':'l'} # 清單的瀏覽模式

"""
展開所有工作清單，後續將依序開始爬蟲
"""
# 這裡會透過Selenium打開一個瀏覽器並開始跑程式~
url = requests.get('https://www.104.com.tw/jobs/search/?' , my_params, headers = headers).url
driver = webdriver.Chrome("./chromedriver")
driver.get(url)

# 網頁的設計方式是滑動到下方時，會自動加載新資料，在這裡透過程式送出Java語法幫我們執行「滑到下方」的動作
for i in range(20): 
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.6)
    
# 自動加載只會加載15次，超過之後必須要點選「手動載入」的按鈕才會繼續載入新資料（可能是防止爬蟲）
k = 1
while k != 0:
    try:
        # 手動載入新資料之後會出現新的more page，舊的就無法再使用，所以要使用最後一個物件[-1]
        driver.find_elements_by_class_name("js-more-page",)[-1].click() 
        # 如果真的找不到，也可以直接找中文!
        # driver.find_element_by_xpath("//*[contains(text(),'手動載入')]").click()
        print('Click 手動載入，' + '載入第' + str(15 + k) + '頁')
        k = k+1
        time.sleep(1) # 時間設定太短的話，來不及載入新資料就會跳錯誤
    except:
        k = 0
        print('No more Job')

# 透過BeautifulSoup解析資料，並先得知有幾筆資料
soup = BeautifulSoup(driver.page_source, 'html.parser')
List = soup.findAll('a',{'class':'js-job-link'})
print('共有 ' + str(len(List)) + ' 筆資料')

"""
解析爬蟲資料並整理成DataFrame
"""
#在正式開始爬蟲之前，我預先定義一個函數，專於用來處理「職務類別」這個複選題，稍後將用這個函數將其串接在一起
# ex: 其他專案管理師、市場調查／市場分析
def bind(cate):
    k = []
    for i in cate:
        if len(i.text) > 0:
            k.append(i.text)
    return str(k)

# 開始逐筆爬資料囉!
JobList = pd.DataFrame()

i = 0
while i < len(List):
    # print('正在處理第' + str(i) + '筆，共 ' + str(len(List)) + ' 筆資料')
    content = List[i]
    # 這裡用Try的原因是，有時候爬太快會遭到系統阻擋導致失敗。因此透過這個方式，當我們遇到錯誤時，會重新再爬一次資料！
    try:
        resp = requests.get('https://' + content.attrs['href'].strip('//'))  #進入每個職缺各別頁面
        soup2 = BeautifulSoup(resp.text,'html.parser')  #剖析每個職缺各別頁面
        df = pd.DataFrame(
            data = [{
                '公司名稱':soup2.find('a', {'class':'cn'}).text,
                '工作職稱':content.attrs['title'],
                '工作內容':soup2.find('p').text,
                '職務類別':bind(soup2.findAll('dd', {'class':'cate'})[0].findAll('span')), #回到上方函數bind(cate)處理
                '工作待遇':soup2.find('dd', {'class':'salary'}).text.split('\n\n',2)[0].replace(' ',''),
                '工作性質':soup2.select('div > dl > dd')[2].text,
                '上班地點':soup2.select('div > dl > dd')[3].text.split('\n\n',2)[0].split('\n',2)[1].replace(' ',''),
                '管理責任':soup2.select('div > dl > dd')[4].text,
                '出差外派':soup2.select('div > dl > dd')[5].text,
                '上班時段':soup2.select('div > dl > dd')[6].text,
                '休假制度':soup2.select('div > dl > dd')[7].text,
                '可上班日':soup2.select('div > dl > dd')[8].text,
                '需求人數':soup2.select('div > dl > dd')[9].text,
                '接受身份':soup2.select('div.content > dl > dd')[10].text,
                '學歷要求':soup2.select('div.content > dl > dd')[12].text,
                '工作經歷':soup2.select('div.content > dl > dd')[11].text,
                '語文條件':soup2.select('div.content > dl > dd')[14].text,
                '擅長工具':soup2.select('div.content > dl > dd')[15].text,
                '工作技能':soup2.select('div.content > dl > dd')[16].text,
                '其他條件':soup2.select('div.content > dl > dd')[17].text,
                '公司福利':soup2.select('div.content > p')[1].text,
                '科系要求':soup2.select('div.content > dl > dd')[13].text,
                '聯絡方式':soup2.select('div.content')[3].text.replace('\n',''),
                '連結路徑':'https://' + content.attrs['href'].strip('//')}],
            columns = ['公司名稱','工作職稱','工作內容','職務類別','工作待遇','工作性質','上班地點','管理責任','出差外派',
                       '上班時段','休假制度','可上班日','需求人數','接受身份','學歷要求','工作經歷','語文條件','擅長工具',
                       '工作技能','其他條件','公司福利','科系要求','聯絡方式','連結路徑'])
        JobList = JobList.append(df, ignore_index=True)
        i += 1
        print("Success and Crawl Next 目前正在爬第" + str(i) + "個職缺資訊")
        time.sleep(0.5) # 執行完休息0.5秒，避免造成對方主機負擔
    except:
        print("Fail and Try Again!")
        pass   #失敗的話Try Again會一直無限迴圈，所以新增跳過再繼續
        
    

JobList

JobList.to_excel('JobList_DA2.xlsx', encoding='cp950')

In [ ]:
104_v2
先前曾發過一篇透過 Selenium 的版本 爬蟲_104人力銀行工作清單 ，但隨著爬蟲技術的提升，就想要來更新一下爬蟲的程式 xD
註：本篇文章僅供研究使用，請勿惡意大量爬取資料造成對方公司的負擔

In [ ]:
[爬蟲案例分享]

難點：

1.資料量大，但每個查詢最多150個分頁，因此需要將查詢的範圍限縮。 解決方式是在查詢時組合地區與職務類型，進行細緻的查詢。

2.雖然幾乎沒有設定反爬蟲的機制，但每個 request 送的速度太快仍然會有失敗的狀況發生 用while迴圈，當request成功是繼續前往下一頁爬蟲，request成功但資料小於20筆，查詢下一個組合，request等於20，表示還有資料需要繼續抓下一個分頁的資料。

3.因為資料量大，建議按照組別存一次資料，這樣遇到錯誤時才不用重新爬資料。

因為查詢職缺最多有20*150=3000筆資料，所以要將查詢的的條件透過條件進行篩選、排列組合，藉此撈出完整的資料
由公司資訊：地址，聯絡電話、職務
縣市和職務的分類有個資的代號，但中間有經過變(如縣市合併)更並不是單純的序列，因此要花時間找一下
可以看各個縣市的職缺數量與職務類型，進一步分析需要什麼技能
你要在哪裡，需要什麼技能、待遇如何
可以去串各縣市的人口數、教育程度等資訊
縣市的產業結構
可行的分析項目

In [ ]:
104人力銀行 爬蟲要點
每次查詢職缺的結果最多回傳20筆*150個分頁的資料量，也就是說當我們查詢的條件超過3000筆時更多的資料就會被截掉。怎麼樣才能抓完完整的資料呢?
解決方式是透過條件來將查詢的結果細緻化，例如按照區域、職務類型將查詢結果細分，最後再進行合併
因為資料量相當多，如果要抓完完整的資料會需要約1整天的時間，那麼一次抓不完的話要怎麼辦呢?
一種解決方式是用多線程的方式來爬資料，加速爬蟲的效率
如果是用一般的方式爬蟲的話，可以將爬取的結果分階段保存資料，而下次爬蟲時只需要繼續爬未完成的資料即可
承1，由於我們將查詢結果按照地區與職務進行細分，大多數組合並不會都有150個分頁的資料量，如果將每個組合都送 150 次 request 無疑會浪費相當多時間，怎麼樣可以更有效率呢?
解決方式是檢查每次查詢的結果，如果回傳20筆職缺，表示下一個分頁還有資料，但如果小於20個分頁就表示後面沒有資料了
雖然 104 沒有設定反爬蟲機制，但是當 request 的速度太快時仍偶爾會有連線失敗的情況發生，但我們不想因此就讓每次 request 都 sleep 一次而降低爬蟲的效率，這時候該如何處理呢?
解決方式是透過 try-except，當 try 成功時才繼續前往下一個分頁，如果失敗了就將同一頁的網址再送一次 request 取資料

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
import os
from IPython.display import clear_output

In [ ]:
#出於習慣就順手加上 user-agents了

headers = {'User-Agent':'GoogleBot'}

In [ ]:
細分查詢結果
同第 1 個爬蟲要點所述，因為資料量龐大，我們需要將查詢結果細分才能確保抓到完整的資料，而在這裡細分的方式是依照地區代碼和職務類型進行查詢。

地區代碼

In [ ]:
url = 'https://static.104.com.tw/category-tool/json/Area.json'
resp = requests.get(url)
df1 = []
for i in resp.json()[0]['n']:
    ndf = pd.DataFrame(i['n'])
    ndf['city'] = i['des']
    df1.append(ndf)
df1=pd.concat(df1, ignore_index=True)
df1 = df1.loc[:,['city','des','no']]
df1 = df1.sort_values('no')
df1

In [ ]:
職務代碼

In [ ]:

url= 'https://static.104.com.tw/category-tool/json/JobCat.json'
resp = requests.get(url)
df2 = []
for i in resp.json():
    for j in i['n']:
        ndf = pd.DataFrame(j['n'])
        ndf['des1'] = i['des']# 職務大分類
        ndf['des2'] = j['des']# 職務小分類
        df2.append(ndf)
df2 = pd.concat(df2, ignore_index=True)
df2 = df2.loc[:,['des1', 'des2', 'des', 'no']]
df2 = df2.sort_values('no')
df2

In [ ]:
讀取先前的爬取結果
同第 2 個爬蟲要點所述，如果是第一次執行程式這段語法可以不用執行，而如果先前有保存結果的話會在這裡偵測先前爬過哪些資料

In [ ]:
tmp = pd.DataFrame([re.sub('\.pkl','',file)for file in os.listdir('./data')],columns=['no'])
df1 = pd.merge(df1, tmp, how='left',on='no',indicator=True)
df1 = df1.loc[df1['_merge']!='both',:]
df1

In [ ]:
爬取職缺資料
這裡是處理第 3 和第 4 點的爬蟲要點，因為不會每個查詢組合都有 150 個分頁的資料，因此需要偵測 request 回來幾筆職缺的資料，等於 20 筆就繼續爬下一個分頁的資料；而小於 20 筆資料則打破迴圈轉去爬下一個組合的職缺資料。
而 request 太快時偶爾會失敗，因此需要用 try-except 函數，當 request 失敗就重新再爬一次資料
最後就是要從 request 回來的結果做一些解析，讓我們從半結構化的資料中萃取成結構化的資料，因為是比較瑣碎的定位 elemnet 在這裡就不多說明了!

In [ ]:
columns = ['公司名稱','公司編號','公司類別','公司類別描述', '公司連結','職缺名稱','職務性質','職缺大分類', '職缺中分類','職缺小分類', '職缺編號', '職務內容','更新日期', '職缺連結', '標籤','公司地址','地區','經歷','學歷']

for areades, areacode in zip(df1['des'],df1['no']):
    values = []
    for jobdes1, jobdes2, jobdes, jobcode in zip(df2['des1'], df2['des2'], df2['des'], df2['no']):
        print(areades, ' | ', jobdes1, ' - ', jobdes2, ' - ' ,jobdes)
        page = 1
        while page <150:
            try:
                url = 'https://www.104.com.tw/jobs/search/?ro=0&jobcat={}&jobcatExpansionType=1&area={}&order=11&asc=0&page={}&mode=s&jobsource=2018indexpoc'.format(jobcode, areacode, page)
                print(url)
                resp = requests.get(url,headers=headers)
                soup = BeautifulSoup(resp.text)
                soup2 = soup.find('div',{'id':'js-job-content'}).findAll('article',{'class':'b-block--top-bord job-list-item b-clearfix js-job-item'})
                print(len(soup2))

                for job in soup2:
                                        
                    update_date = job.find('span',{'class':'b-tit__date'}).text
                    update_date = re.sub('\r|\n| ','',update_date)

                    try:
                        address = job.select('ul > li > a')[0]['title']
                        address = re.findall('公司住址：(.*?)$',address)[0]
                    except:
                        address = ''
                   
                    loc = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[0].text
                    exp = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[1].text
                    try:
                        edu = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[2].text
                    except:
                        edu = ''
                    
                    try:
                        content = job.find('p').text
                    except:
                        content = ''
                    try:
                        tags = [tag.text for tag in soup2[0].find('div',{'class':'job-list-tag b-content'}).findAll('span')]
                    except:
                        tags = []
                    
                    
                    value = [job['data-cust-name'], # 公司名稱
                             job['data-cust-no'], # 公司編號
                             job['data-indcat'], # 公司類別
                             job['data-indcat-desc'], # 公司類別描述
                             job.select('ul > li > a')[0]['href'], # 公司連結
                             job['data-job-name'],# 職缺名稱
                             job['data-job-ro'], # 職務性質 _判斷全職兼職 1全職/2兼職/3高階/4派遣/5接案/6家教
                             jobdes1, # 職缺大分類
                             jobdes2, # 職缺中分類
                             jobdes, # 職缺小分類
                             job['data-job-no'],# 職缺編號
                             content, # 職務內容
                             update_date, # 更新日期
                             job.find('a',{'class':'js-job-link'})['href'], # 職缺連結
                             tags, # 標籤
                             address,# 公司地址
                             loc, # 地區
                             exp,# 經歷
                             edu  # 學歷
                            ]
                    values.append(value)
                
                page+=1
                print(len(values))
                if len(soup2) < 20:
                    break
            except:
                print('Retry')
        
    df = pd.DataFrame()
    df = pd.DataFrame(values, columns=columns)
    df.to_pickle('./data/' + areacode + '.pkl')
    clear_output()
    print('===================================  Save Data  ===================================')

In [ ]:
組合爬蟲結果
由於先前將職缺資訊分成許多小查詢來抓資料並存成不同的檔案，因此在這裡我們就寫個簡單的迴圈來讀取與合併資料吧!

In [ ]:
df = []
for i in os.listdir('./data/'):
    ndf = pd.read_pickle('./data/' + i)
    df.append(ndf)
df = pd.concat(df, ignore_index=True)
df.info()

In [ ]:
後記
有了這些資料我們就可以進行許多有價值的分析囉，包含各縣市的職缺數量、產業結構差異，並且可以進一步幫助求職者們在選擇工作時可以評估自己個興趣與期望的工作地點等等!